In [1]:
import numpy as np
import pandas as pd
import os
import pickle

from IPython.display import display
cwd = os.getcwd()

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
from scipy import stats
from scipy.optimize import curve_fit
import os
import re

cwd = os.getcwd()
print (cwd)

fdat = open(cwd+'\\TCA_PPP_Glycolysis_noFBA.dat', 'r')

left ='LEFT'
right = 'RIGHT'
left_compartment = 'LEFT_COMPARTMENT'
right_compartment = 'RIGHT_COMPARTMENT'
enzyme_level = 'ENZYME_LEVEL'
deltag0 = 'DGZERO'
deltag0_sigma = 'DGZERO StdDev'
same_compartment = 'Same Compartment?'
full_rxn = 'Full Rxn'
FLUX='FLUX'
reactions = pd.DataFrame(index=[],columns=[left, right, left_compartment, right_compartment, enzyme_level, deltag0, deltag0_sigma, same_compartment,full_rxn,FLUX])
reactions.index.name='REACTION'
S_matrix = pd.DataFrame(index=[],columns=[enzyme_level])
S_matrix.index.name='REACTION'

for line in fdat:
    if (line.startswith('REACTION')):
        rxn_name = line[9:-1].lstrip()
        S_matrix.loc[rxn_name,enzyme_level] = 1.0
        reactions.loc[rxn_name,enzyme_level] = 1.0

    if (re.match("^LEFT\s",line)):
        line = line.upper()
        left_rxn = line[4:-1].lstrip()
        left_rxn = re.sub(r'\s+$', '', left_rxn) #Remove trailing white space
        reactions.loc[rxn_name,left] = left_rxn

    elif (re.match('^RIGHT\s',line)):
        line = line.upper()
        right_rxn = line[5:-1].lstrip()
        right_rxn = re.sub(r'\s+$', '', right_rxn) #Remove trailing white space
        reactions.loc[rxn_name,right] = right_rxn
        
    elif (line.startswith(left_compartment)):
        cpt_name = line[16:-1].lstrip()
        reactions.loc[rxn_name,left_compartment] = cpt_name
        reactants = re.split(' \+ ',left_rxn)
        for idx in reactants:
            values = re.split(' ', idx);
            if len(values) == 2:
                stoichiometry = np.float64(values[0]);
                molecule = values[1];
                if not re.search(':',molecule):
                    molecule = molecule + ':' + cpt_name
            else:
                stoichiometry = np.float64(-1.0);
                molecule = values[0]; 
                if not re.search(':',molecule):
                    molecule = molecule + ':' + cpt_name
            S_matrix.loc[rxn_name,molecule] = stoichiometry;


    elif (line.startswith(right_compartment)):
        cpt_name = line[17:-1].lstrip()
        reactions.loc[rxn_name,right_compartment] = cpt_name
        products = re.split(' \+ ',right_rxn)
        for idx in products:
            values = re.split(' ', idx);
            if len(values) == 2:
                stoichiometry = np.float64(values[0]);
                molecule = values[1];
                if not re.search(':',molecule):
                    molecule = molecule + ':' + cpt_name
            else:
                stoichiometry = np.float64(1.0);
                molecule = values[0];
                if not re.search(':',molecule):
                    molecule = molecule + ':' + cpt_name
            S_matrix.loc[rxn_name,molecule] = stoichiometry;

    elif (re.match("^ENZYME_LEVEL\s", line)):
        level = line[12:-1].lstrip()
        reactions.loc[rxn_name,enzyme_level] = float(level)
        S_matrix.loc[rxn_name,enzyme_level] = float(level)
                
    elif re.match('^COMMENT',line):
        continue
    elif re.match(r'//',line):
        continue
    elif re.match('^#',line):
        continue
        
fdat.close()
S_matrix.fillna(0,inplace=True)
S_active = S_matrix[S_matrix[enzyme_level] > 0.0]
active_reactions = reactions[reactions[enzyme_level] > 0.0]
del S_active[enzyme_level]
# Delete any columns/metabolites that have all zeros in the S_mat matrix:
S_active = S_active.loc[:, (S_active != 0).any(axis=0)]
np.shape(S_active.values)
reactions[full_rxn] = reactions[left] + ' = ' + reactions[right]

use_experimental_data=True
use_low_high=False #False means High/High

conc = 'Conc'
variable = 'Variable'
conc_exp = 'Conc_Experimental'
metabolites = pd.DataFrame(index = S_active.columns, columns=[conc,conc_exp,variable])
metabolites[conc] = 0.001
metabolites[variable] = True


#%%
nvariables = metabolites[metabolites[variable]].count()
nvar = nvariables[variable]

metabolites.sort_values(by=variable, axis=0,ascending=False, inplace=True,)
#display(metabolites)


# ## Prepare model for optimization

# - Adjust S Matrix to use only reactions with activity > 0, if necessary.
# - Water stoichiometry in the stiochiometric matrix needs to be set to zero since water is held constant.
# - The initial concentrations of the variable metabolites are random.
# - All concentrations are changed to log counts.
# - Equilibrium constants are calculated from standard free energies of reaction.
# - R (reactant) and P (product) matrices are derived from S.

# Make sure all the indices and columns are in the correct order:
active_reactions = reactions[reactions[enzyme_level] > 0.0]
#display(reactions)
display(metabolites.index)
Sactive_index = S_active.index

active_reactions.reindex(index = Sactive_index, copy = False)
#%% Make dictionary with metabolites indices and BiGG metabolite abbreviates so Escher software can read the json correctly

#display(metabolites.index)


bigg_reaction_abbr_p3 = pd.Series(['CSM','ACONT','ICDH','AKGD','SUCOAS',
        'SUCD','FUM','MDH','GAPD','PGK','TPI','FBA','PYK','PGM','ENO',
        'HEX1','PGI','PFK','PYRt2M','PDH','G6PDH','PGL','GND','RPE','RPI',
        'TKT1','TALA','TKT2','GOGAT'])

reactions.index=bigg_reaction_abbr_p3
active_reactions.index=bigg_reaction_abbr_p3
S_active.index=bigg_reaction_abbr_p3

display(S_active.index)

C:\Users\samuel_britton\Documents\cannon\Final_Pathways\Python_Notebook_Regulation\plotting


Index(['OXALOACETATE:MITOCHONDRIA', 'NADP+:CYTOSOL',
       'D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL', 'PHOSPHOENOLPYRUVATE:CYTOSOL',
       'PYRUVATE:CYTOSOL', '2-PHOSPHO-D-GLYCERATE:CYTOSOL', 'H2O:CYTOSOL',
       'BETA-D-GLUCOSE:CYTOSOL', 'BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL',
       'D-FRUCTOSE_6-PHOSPHATE:CYTOSOL', 'PYRUVATE:MITOCHONDRIA',
       'D-GLUCONO-1,5-LACTONE_6-PHOSPHATE:CYTOSOL', 'ATP:CYTOSOL',
       'NADPH:CYTOSOL', '6-PHOSPHO-D-GLUCONATE:CYTOSOL',
       'D-RIBULOSE-5-PHOSPHATE:CYTOSOL', 'CO2:CYTOSOL',
       'D-XYLULOSE-5-PHOSPHATE:CYTOSOL', 'D-RIBOSE-5-PHOSPHATE:CYTOSOL',
       'SEDOHEPTULOSE_7-PHOSPHATE:CYTOSOL', 'D-ERYTHROSE-4-PHOSPHATE:CYTOSOL',
       'L-GLUTAMATE:MITOCHONDRIA', 'GLYCERONE_PHOSPHATE:CYTOSOL',
       '3-PHOSPHO-D-GLYCERATE:CYTOSOL', 'ACETYL-COA:MITOCHONDRIA',
       'ADP:MITOCHONDRIA', 'H2O:MITOCHONDRIA', 'CITRATE:MITOCHONDRIA',
       'COA:MITOCHONDRIA', 'ISOCITRATE:MITOCHONDRIA', 'NAD+:MITOCHONDRIA',
       '2-OXOGLUTARATE:MITOCHONDRIA', 'NADH:MIT

Index(['CSM', 'ACONT', 'ICDH', 'AKGD', 'SUCOAS', 'SUCD', 'FUM', 'MDH', 'GAPD',
       'PGK', 'TPI', 'FBA', 'PYK', 'PGM', 'ENO', 'HEX1', 'PGI', 'PFK',
       'PYRt2M', 'PDH', 'G6PDH', 'PGL', 'GND', 'RPE', 'RPI', 'TKT1', 'TALA',
       'TKT2', 'GOGAT'],
      dtype='object')

In [2]:
metabolites = pd.Series(['OXALOACETATE:MITOCHONDRIA', 'NADP+:CYTOSOL',
       'D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL', 'PHOSPHOENOLPYRUVATE:CYTOSOL',
       'PYRUVATE:CYTOSOL', '2-PHOSPHO-D-GLYCERATE:CYTOSOL', 'H2O:CYTOSOL',
       'BETA-D-GLUCOSE:CYTOSOL', 'BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL',
       'D-FRUCTOSE_6-PHOSPHATE:CYTOSOL', 'PYRUVATE:MITOCHONDRIA',
       'D-GLUCONO-1,5-LACTONE_6-PHOSPHATE:CYTOSOL', 'ATP:CYTOSOL',
       'NADPH:CYTOSOL', '6-PHOSPHO-D-GLUCONATE:CYTOSOL',
       'D-RIBULOSE-5-PHOSPHATE:CYTOSOL', 'CO2:CYTOSOL',
       'D-XYLULOSE-5-PHOSPHATE:CYTOSOL', 'D-RIBOSE-5-PHOSPHATE:CYTOSOL',
       'SEDOHEPTULOSE_7-PHOSPHATE:CYTOSOL', 'D-ERYTHROSE-4-PHOSPHATE:CYTOSOL',
       'L-GLUTAMATE:MITOCHONDRIA', 'GLYCERONE_PHOSPHATE:CYTOSOL',
       '3-PHOSPHO-D-GLYCERATE:CYTOSOL', 'ACETYL-COA:MITOCHONDRIA',
       'ADP:MITOCHONDRIA', 'H2O:MITOCHONDRIA', 'CITRATE:MITOCHONDRIA',
       'COA:MITOCHONDRIA', 'ISOCITRATE:MITOCHONDRIA', 'NAD+:MITOCHONDRIA',
       '2-OXOGLUTARATE:MITOCHONDRIA', 'NADH:MITOCHONDRIA', 'CO2:MITOCHONDRIA',
       'SUCCINYL-COA:MITOCHONDRIA', 'ORTHOPHOSPHATE:MITOCHONDRIA',
       'ADP:CYTOSOL', 'SUCCINATE:MITOCHONDRIA', 'ATP:MITOCHONDRIA',
       'FUMARATE:MITOCHONDRIA', '(S)-MALATE:MITOCHONDRIA',
       'D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL', 'ORTHOPHOSPHATE:CYTOSOL',
       'NAD+:CYTOSOL', '3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL',
       'NADH:CYTOSOL', 'L-GLUTAMINE:MITOCHONDRIA'])
    
bigg = pd.Series(['oaa_m', 'nadp_c',
       'fdp_c', 'pep_c',
       'pyr_c', '2pg_c', 'h2o_c',
       'glc__D_c', 'M01389_c',
       'f6p_c', 'pyr_m',
       '6pgl_c', 'atp_c',
       'nadph_c', '6pgc_c',
       'ru5p__D_c', 'co2_c',
       'xu5p__D_c', 'ru5p__D_c',
       's7p_c', 'e4p_c',
       'glu__L_m', 'dhap_c',
       '3pg_c', 'accoa_m',
       'adp_m', 'h2o_m', 'cit_m',
       'coa_m', 'icit_m', 'nad_m',
       'akg_m', 'nadh_m', 'co2_m',
       'succoa_m', 'pi_m',
       'adp_c', 'succ_m', 'atp_m',
       'fum_m', 'mal__L_m',
       'g3p_c', 'pi_c',
       'nad_c', '13dpg_c',
       'nadh_c', 'gln__L_m'])


#{key: value for (key, value) in iterable}
metabolite_bigg = dict(zip(metabolites, bigg))

with open(cwd+'/metabolite_bigg.pickle', 'wb') as handle:
    pickle.dump(metabolite_bigg, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(cwd+'/metabolite_bigg.pickle', 'rb') as handle:
    metabolite_to_bigg = pickle.load(handle)
    
json_model_file = open(cwd+'/iMM904_CentralCarbonMetabolism4_model_file.json', 'w')

if (1):
    print('{\n\"metabolites\":[',file=json_model_file)
    num_metab = len(S_active.columns)
    i = 1
    for m_idx in (S_active.columns):
        print('{\n\"id\":\"%s\"' % (metabolite_to_bigg[m_idx]),file=json_model_file)
        if (i < num_metab):
            print('},',file=json_model_file)
        else:
            print('}',file=json_model_file)
        i=i+1
    print('],\n',file=json_model_file)
    num_rxns = len(reactions.index)
    i = 1
    print('\"reactions\":[',file=json_model_file)
    for idx in reactions.index:
        print('{\n\"id\":\"%s\",' % (idx),file=json_model_file)
        print('\"metabolites\":{',file=json_model_file)
        rxn_metabs = {}
        for m_idx in (S_active.columns):
            if(S_active.loc[idx,m_idx] != 0):
                
                rxn_metabs[metabolite_to_bigg[m_idx]] = S_active.loc[idx,m_idx]
                #rxn_metabs[](metabolite_to_bigg[m_idx])
        key_list =[*rxn_metabs]
        nkeyz = len(key_list)
        for ii in key_list[:-1]:
            print('\"%s\":%d,'% (ii,rxn_metabs[ii]),file=json_model_file)
        print('\"%s\": %d'% (key_list[-1],rxn_metabs[key_list[-1]]),file=json_model_file)
        print('}',file=json_model_file)
        if (i < num_rxns):
            print('},',file=json_model_file)
        else:
            print('}',file=json_model_file)
        i=i+1
    print('],',file=json_model_file)
    print('\"genes\":[',file=json_model_file)
    print('],',file=json_model_file)
    print('\"id\":\"iMM904_BC\",',file=json_model_file)

    print('\"compartments\":{',file=json_model_file)
    print('\"c\":\"cytosol\",',file=json_model_file)
    print('\"e\":\"extracellular space\",',file=json_model_file)
    print('\"m\":\"mitochondria\"',file=json_model_file)
    print('},',file=json_model_file)
    print('\"version\":\"1\"\n}',file=json_model_file)
json_model_file.close()
print(cwd)

C:\Users\samuel_britton\Documents\cannon\Final_Pathways\Python_Notebook_Regulation\plotting


In [11]:
import json
#{key: value for (key, value) in iterable}
metabolite_bigg = dict(zip(metabolites, bigg))

with open(cwd+'/metabolite_bigg.pickle', 'wb') as handle:
    pickle.dump(metabolite_bigg, handle, protocol=pickle.HIGHEST_PROTOCOL)


bigg_reorder_p3 = [15, 16, 17, 11, 10, 8, 9, 13, 14, 12, 18, 19, 
                   20, 21, 22, 24, 23, 25, 26, 27,
                   0, 1, 2, 3, 4, 5, 6, 7, 28]
bigg_reaction_abbr_p3[bigg_reorder_p3]

#Local MCA for Low/High
#Unrerstricted MCA for High/High
#RL solution for High/High, no PFK
flux_high_low = np.loadtxt(cwd+'/data/flux_experiment_method1_highlow.txt', dtype=float)
flux_high_high = np.loadtxt(cwd+'/data/flux_experiment_method2_highhigh.txt', dtype=float)
flux_high_high_nopfk = np.loadtxt(cwd+'/data/flux_experiment_RL_highhigh_PFKZERO.txt', dtype=float)
activity_methodRL_noconstraint_experimental_path3 = np.loadtxt(cwd+'/data/activities_experiment_RL_noconstraint_highhigh.txt', dtype=float)

#Use one of the loaded fluxes or activities from /data to plot with escher
active_reactions['FLUX'] = flux_high_low

display(active_reactions.index)


#flux_dictionary = dict(zip(S_active.index, flux_methodRL_noconstraint_experimental_path3))
#with open(cwd+'/tca_ppp_glycolysis_ml_regulated_flux_data.json', 'w') as f:
#    json.dump(flux_dictionary, f)

#activity_dictionary = dict(zip(S_active.index, activity_methodRL_noconstraint_experimental_path3))
#with open(cwd+'/tca_ppp_glycolysis_ml_regulated_activity_data.json', 'w') as f:
#    json.dump(activity_dictionary, f)

Index(['CSM', 'ACONT', 'ICDH', 'AKGD', 'SUCOAS', 'SUCD', 'FUM', 'MDH', 'GAPD',
       'PGK', 'TPI', 'FBA', 'PYK', 'PGM', 'ENO', 'HEX1', 'PGI', 'PFK',
       'PYRt2M', 'PDH', 'G6PDH', 'PGL', 'GND', 'RPE', 'RPI', 'TKT1', 'TALA',
       'TKT2', 'GOGAT'],
      dtype='object')

### PLOT FLUX


In [12]:

from IPython.display import display
from escher import Builder

reaction_data = active_reactions['FLUX'].to_dict()

b = Builder(map_json='iMM904_CentralCarbonMetabolism4_nopfk3_nofba3.json', reaction_data=reaction_data)
display(b)


Builder(reaction_data={'CSM': 12.671170363109077, 'ACONT': 12.671170363109118, 'ICDH': 12.671170363109056, 'AK…